# Converting calibrated data from MARS+ROOT to csv and/or parquet

**ST0317**

Overall simulation parameters:
- spectral slope -1.6
- min energy = 10 GeV
- max energy = 30 TeV
- gammas [particle ID 1]

In [ ]:
import dotenv

# load in environment variables
dotenv.load_dotenv()

import datetime as dt
import sys
from pathlib import Path

import magicpy.analysis as analysis
import magicpy.datalevel as dl
import magicpy.moons as moons
import magicpy.periods as periods
import magicpy.zeniths as zeniths
import numpy as np
import pandas as pd
import uproot

sys.path.append(str(Path(".").absolute().parent / "magicdl"))
import convert

In [2]:
mc = analysis.MC(
    period=periods.ST0317,
    zenith=zeniths.za05to35,
)

Loading existing analysis from /remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0317/MC_ST0317_za05to35_None.magicpy
INFO: RFs not found for: ST0317 za05to35 weak moon.
INFO: RFs not found for: ST0317 za05to35 moderate moon.
INFO: RFs not found for: ST0317 za05to35 decent moon.
INFO: RFs not found for: ST0317 za05to35 strong moon.
INFO: RFs not found for: ST0317 za05to35 high moon.
INFO: RFs not found for: ST0317 za05to35 super moon.
INFO: RFs not found for: ST0317 za05to35 reduced_decent moon.
INFO: RFs not found for: ST0317 za05to35 reduced_strong moon.
INFO: RFs not found for: ST0317 za05to35 reduced_high moon.


In [8]:
mc.runs.run_number.max()

855617

In [3]:
m1_runs = mc.runs[mc.runs.telescope_number == 1].sort_values("run_number")
m2_runs = mc.runs[mc.runs.telescope_number == 2].sort_values("run_number")

In [4]:
# pull a sample run
run_path_m1 = m1_runs.iloc[0].local_path
run_path_m2 = m2_runs.iloc[0].local_path

### Exploring the data

In [5]:
# interesting run headers
f = uproot.open(run_path_m1)
print(f.keys())

# have a length of 1000?  so even non-triggered events are stored?
original = [
    "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fEnergy",
    "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fImpact",
    "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fTelescopePhi",
    "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fTelescopeTheta",
]

headers = [
    "RunHeaders;1/MRawRunHeader./MRawRunHeader.fTelescopeNumber", 
    # "RunHeaders;1/MMcRunHeader./MMcRunHeader.fNumEvents",  # this is num simulated, not num triggered
    "RunHeaders;1/MRawRunHeader./MRawRunHeader.fRunNumber",
]

# events
event_info = [
    'Events;1/MMcEvt./MMcEvt.fEvtNumber',
    'Events;1/MMcEvt./MMcEvt.fEnergy',
    'Events;1/MMcEvt./MMcEvt.fZFirstInteraction',
    'Events;1/MMcEvt./MMcEvt.fTheta',
    'Events;1/MMcEvt./MMcEvt.fPhi',
    'Events;1/MMcEvt./MMcEvt.fTelescopeTheta',
    'Events;1/MMcEvt./MMcEvt.fTelescopePhi',
    'Events;1/MMcEvt./MMcEvt.fCoreD',
    'Events;1/MMcEvt./MMcEvt.fCoreX',
    'Events;1/MMcEvt./MMcEvt.fCoreY',
    'Events;1/MMcEvt./MMcEvt.fImpact',
    'Events;1/MMcEvt./MMcEvt.fTimeFirst',
    'Events;1/MMcEvt./MMcEvt.fTimeLast',
    # p.e. image data
    'Events;1/MCerPhotEvt./MCerPhotEvt.fPixels/MCerPhotEvt.fPixels.fPhot',
    # what are these two?
    'Events;1/MPedPhotFundamental./MPedPhotFundamental.fArray/MPedPhotFundamental.fArray.fMean',
    'Events;1/MPedPhotFundamental./MPedPhotFundamental.fArray/MPedPhotFundamental.fArray.fRms',
]

arrival_time = [
    # arrival time
    # must read with np.array(t.tojson()) for each entry
    'Events;1/MArrivalTime./MArrivalTime.fData',
]

['OriginalMC;1', 'RunTails;1', 'RunHeaders;1', 'Events;1']


In [11]:
# scan the file for all keys

import uproot.exceptions

for key in f.keys():
    
    print(f"\n----\nKEY {key}\n")

    for k, v in f[key].items():
        try:
            array = v.array(library="np")
            # check if you can index the array
            try:
                length = len(array)
            except TypeError:
                length = ".."
            try:
                shape = array.shape, array[0].shape
            except AttributeError:
                shape = ".."
            try:
                n_unique = len(np.unique(array))
            except TypeError:
                n_unique = ".."
            print(f"    {key}/{k}")
            print(f"        l=[{length}] // n_u={n_unique} // s={shape} // t={type(array)} // v0={array[0]}")
        except (KeyError, TypeError, ValueError, uproot.DeserializationError):
            if hasattr(v, "keys"):
                print()
                print(f"    {key}/{k}" + f"\n        KEYS: {len(v.keys())}")
            else:
                print(f"    {key}/{k}" + f"\n        {key}{v}")


----
KEY OriginalMC;1

    OriginalMC;1/MMcEvtBasic.

    OriginalMC;1/MMcEvtBasic.
        KEYS: 9
    OriginalMC;1/MMcEvtBasic./MMcEvtBasic.MParContainer

    OriginalMC;1/MMcEvtBasic./MMcEvtBasic.MParContainer
        KEYS: 3
    OriginalMC;1/MMcEvtBasic./MMcEvtBasic.MParContainer/MMcEvtBasic.MParContainer.TObject

    OriginalMC;1/MMcEvtBasic./MMcEvtBasic.MParContainer/MMcEvtBasic.MParContainer.TObject
        KEYS: 2
    OriginalMC;1/MMcEvtBasic./MMcEvtBasic.MParContainer/MMcEvtBasic.MParContainer.TObject/MMcEvtBasic.MParContainer.fUniqueID
        l=[1000] // n_u=1 // s=((1000,), ()) // t=<class 'numpy.ndarray'> // v0=0
    OriginalMC;1/MMcEvtBasic./MMcEvtBasic.MParContainer/MMcEvtBasic.MParContainer.TObject/MMcEvtBasic.MParContainer.fBits
        l=[1000] // n_u=1 // s=((1000,), ()) // t=<class 'numpy.ndarray'> // v0=50331648
    OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fPartId
        l=[1000] // n_u=1 // s=((1000,), ()) // t=<class 'numpy.ndarray'> // v0=1
    OriginalMC;1/MMcEv

In [196]:
overall_params = {}

for k in original:
    title = k.split(".")[-1]
    overall_params[title] = f[k].array()[0]
    
for k in headers:
    title = k.split(".")[-1]
    overall_params[title] = f[k].array()[0]
    
event_params = {}

for k in event_info:
    title = k.split(".")[-1]
    event_params[title] = f[k].array().to_numpy()
    
n_events = len(event_params["fEvtNumber"])
overall_params["fNumTrig"] = n_events


arrival_times = []
timing_data = f["Events;1/MArrivalTime./MArrivalTime.fData"].array()
for t in timing_data:
    d = t.to_list()
    arrival_times.append(d)

event_params["MArrivalTime"] = np.array(arrival_times).reshape(n_events, len(d))

# generate event ID from telescope number, run number, and event number
event_ids = []

for i in range(n_events):
    event_ids.append(f"{overall_params['fTelescopeNumber']}.{overall_params['fRunNumber']}.{event_params['fEvtNumber'][i]}")

event_params["event_id"] = np.array(event_ids)

print("--- Overall params: ---")
for k, v in overall_params.items():
    print(k, v)

print("\n--- Event params: ---")
for k, v in event_params.items():
    print(k, v.shape)

--- Overall params: ---
fEnergy 11.203301
fImpact 26627.633
fTelescopePhi 0.6726506
fTelescopeTheta 0.33419406
fTelescopeNumber 1
fRunNumber 821318
fNumTrig 51

--- Event params: ---
fEvtNumber (51,)
fEnergy (51,)
fZFirstInteraction (51,)
fTheta (51,)
fPhi (51,)
fTelescopeTheta (51,)
fTelescopePhi (51,)
fCoreD (51,)
fCoreX (51,)
fCoreY (51,)
fImpact (51,)
fTimeFirst (51,)
fTimeLast (51,)
fPhot (51, 1183)
fMean (51, 1183)
fRms (51, 1183)
MArrivalTime (51, 1183)
event_id (51,)


# Extracting the images

In [4]:
def extract_mc_events(filepath: Path | str, verbose: bool = False):

    filepath = Path(filepath).absolute()
    
    # interesting run headers

    # have a length of 1000?  so even non-triggered events are stored?
    original = [
        # "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fEnergy",
        # "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fImpact",
        # "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fTelescopePhi",
        # "OriginalMC;1/MMcEvtBasic./MMcEvtBasic.fTelescopeTheta",
    ]

    headers = [
        "RunHeaders;1/MRawRunHeader./MRawRunHeader.fTelescopeNumber", 
        # "RunHeaders;1/MMcRunHeader./MMcRunHeader.fNumEvents",  # this is num simulated, not num triggered
        "RunHeaders;1/MRawRunHeader./MRawRunHeader.fRunNumber",
    ]

    # events
    event_info = [
        'Events;1/MMcEvt./MMcEvt.fEvtNumber',
        'Events;1/MMcEvt./MMcEvt.fEnergy',
        'Events;1/MMcEvt./MMcEvt.fZFirstInteraction',
        'Events;1/MMcEvt./MMcEvt.fTheta',
        'Events;1/MMcEvt./MMcEvt.fPhi',
        'Events;1/MMcEvt./MMcEvt.fTelescopeTheta',
        'Events;1/MMcEvt./MMcEvt.fTelescopePhi',
        # 'Events;1/MMcEvt./MMcEvt.fCoreD',
        # 'Events;1/MMcEvt./MMcEvt.fCoreX',
        # 'Events;1/MMcEvt./MMcEvt.fCoreY',
        # 'Events;1/MMcEvt./MMcEvt.fImpact',
        # 'Events;1/MMcEvt./MMcEvt.fTimeFirst',
        # 'Events;1/MMcEvt./MMcEvt.fTimeLast',
        # p.e. image data
        'Events;1/MCerPhotEvt./MCerPhotEvt.fPixels/MCerPhotEvt.fPixels.fPhot',
        # what are these two?
        # 'Events;1/MPedPhotFundamental./MPedPhotFundamental.fArray/MPedPhotFundamental.fArray.fMean',
        # 'Events;1/MPedPhotFundamental./MPedPhotFundamental.fArray/MPedPhotFundamental.fArray.fRms',
    ]
    
    with uproot.open(filepath) as f:
        
        overall_params = {}

    for k in original:
        title = k.split(".")[-1]
        overall_params[title] = f[k].array()[0]
        
    for k in headers:
        title = k.split(".")[-1]
        overall_params[title] = f[k].array()[0]
        
    event_params = {}

    for k in event_info:
        title = k.split(".")[-1]
        event_params[title] = f[k].array().to_numpy()
        
    n_events = len(event_params["fEvtNumber"])
    overall_params["fNumTrig"] = n_events


    arrival_times = []
    timing_data = f["Events;1/MArrivalTime./MArrivalTime.fData"].array()
    for t in timing_data:
        d = t.to_list()
        arrival_times.append(d)

    event_params["MArrivalTime"] = np.array(arrival_times).reshape(n_events, len(d))
    
    # generate event ID from run number and event number
    event_ids = []

    for i in range(n_events):
        event_ids.append(f"{overall_params['fRunNumber']}.{event_params['fEvtNumber'][i]}")

    event_params["event_id"] = np.array(event_ids)

    if verbose:
        print(f"Telescope {overall_params['fTelescopeNumber']} / Run {overall_params['fRunNumber']}")
        print(f"Extracted {n_events} events from {filepath}")
        print("--- Overall params: ---")
        for k, v in overall_params.items():
            print(k, v)

        print("\n--- Event params: ---")
        for k, v in event_params.items():
            print(k, v.shape)
        print()
        
    # convert ndarrays to lists
    for k, v in event_params.items():
        if isinstance(v, np.ndarray):
            event_params[k] = v.tolist()
            
    return overall_params, event_params

## Stereo matching

In [5]:
def merge_data(
    m1_filepath: Path | str,
    m2_filepath: Path | str,
    stereo_only: bool = True,
    export: bool = False,
    export_filepath: Path | None = None,
    verbose: bool = False,
):
    
    m1_filepath = Path(m1_filepath).absolute()
    m2_filepath = Path(m2_filepath).absolute()
        
    
    o1, e1 = extract_mc_events(run_path_m1, verbose=verbose)
    o2, e2 = extract_mc_events(run_path_m2, verbose=verbose)
    
    # turn each into a dataframe
    df1 = pd.DataFrame(e1)

    # add fTelecopeNumber and fRunNumber to each event
    # df1['fTelescopeNumber'] = o1['fTelescopeNumber']
    df1['fRunNumber'] = o1['fRunNumber']

    df2 = pd.DataFrame(e2)

    # add fTelecopeNumber and fRunNumber to each event
    # df2['fTelescopeNumber'] = o2['fTelescopeNumber']
    df2['fRunNumber'] = o2['fRunNumber']

    # drop all rows with event number == 1000
    # for some reason 1000 is always glitchy
    df1 = df1[df1.fEvtNumber != 1000]
    df2 = df2[df2.fEvtNumber != 1000]
    
    merge = pd.merge(
        df1, 
        df2, 
        on=[
            'event_id', 
            'fEvtNumber',
            'fRunNumber',
            'fEnergy',
            'fTheta',
            'fPhi',
            'fTelescopeTheta',
            'fTelescopePhi',
            'fZFirstInteraction',
        ], 
        suffixes=('M1', 'M2'),
        how="inner" if stereo_only else "outer",
    )

    # order the columns
    all_cols = list(merge.columns)
    first_cols = [
        'event_id',
        'fRunNumber',
        'fEvtNumber',
        'fEnergy',
        'fTheta',
        'fPhi',
        'fTelescopeTheta',
        'fTelescopePhi',
        'fZFirstInteraction',
    ]

    merge = merge[first_cols + [c for c in all_cols if c not in first_cols]]
    
    if export and export_filepath is None:
        
        run_number = o1['fRunNumber']
        file_stem = f"MC_run_{run_number}"
        
        if stereo_only:
            file_stem += "_stereo"
        export_filepath = Path(f"./{file_stem}.parquet").absolute()
    
    if export:
        merge.to_parquet(export_filepath)
        return export_filepath
    else:
        return merge

In [18]:
df = merge_data(run_path_m1, run_path_m2, stereo_only=True, export=True)

In [22]:
run_path_m1

PosixPath('/remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0317/za05to35/weak/Calibrated_M1/GA_M1_za05to35_9_821318_Y_w0.root')

In [28]:
o, e = extract_mc_events(Path('/remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0317/za05to35/weak/Calibrated_M1/GA_M1_za05to35_9_821319_Y_w0.root'), verbose=True)

Telescope 1 / Run 821319
Extracted 143 events from /remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0317/za05to35/weak/Calibrated_M1/GA_M1_za05to35_9_821319_Y_w0.root
--- Overall params: ---
fTelescopeNumber 1
fRunNumber 821319
fNumTrig 143

--- Event params: ---
fEvtNumber (143,)
fEnergy (143,)
fZFirstInteraction (143,)
fTheta (143,)
fPhi (143,)
fTelescopeTheta (143,)
fTelescopePhi (143,)
fPhot (143, 1183)
MArrivalTime (143, 1183)
event_id (143,)



In [125]:
d = pd.read_parquet(df)

### Debugging stereo matching

In [90]:
o1, e1 = extract_mc_events(run_path_m1, verbose=False)
o2, e2 = extract_mc_events(run_path_m2, verbose=False)

In [91]:
# turn each into a dataframe
df1 = pd.DataFrame(e1)

# add fTelecopeNumber and fRunNumber to each event
# df1['fTelescopeNumber'] = o1['fTelescopeNumber']
df1['fRunNumber'] = o1['fRunNumber']

df2 = pd.DataFrame(e2)

# add fTelecopeNumber and fRunNumber to each event
# df2['fTelescopeNumber'] = o2['fTelescopeNumber']
df2['fRunNumber'] = o2['fRunNumber']

# drop all rows with event number == 1000
df1 = df1[df1.fEvtNumber != 1000]
df2 = df2[df2.fEvtNumber != 1000]

In [92]:
merge = pd.merge(
    df1, 
    df2, 
    on=[
        'event_id', 
        'fEvtNumber',
        'fRunNumber',
        'fEnergy',
        'fTheta',
        'fPhi',
        'fTelescopeTheta',
        'fTelescopePhi',
        'fZFirstInteraction',
    ], 
    suffixes=('_m1', '_m2'),
    how="outer",
)

# order the columns
all_cols = list(merge.columns)
first_cols = [
    'event_id',
    'fRunNumber',
    'fEvtNumber',
    'fEnergy',
    'fTheta',
    'fPhi',
    'fTelescopeTheta',
    'fTelescopePhi',
    'fZFirstInteraction',
]

merge = merge[first_cols + [c for c in all_cols if c not in first_cols]]

with pd.option_context('display.max_columns', None, 'display.max_rows', None):
    display(merge)

,event_id,fRunNumber,fEvtNumber,fEnergy,fTheta,fPhi,fTelescopeTheta,fTelescopePhi,fZFirstInteraction,fPhot_m1,MArrivalTime_m1,fPhot_m2,MArrivalTime_m2
0,821318.1000,821318,1000,117.193596,0.409536,2.844549,0.412085,2.860828,1532896.500,"[2.296875, 1.92578125, 2.6875, 3.4921875, 2.29...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",NaN,NaN
1,821318.21,821318,21,341.861969,0.518891,2.747146,0.518095,2.733148,1279174.500,"[3.140625, 5.109375, 3.25, 2.3359375, 2.0625, ...","[28.1875, 27.625, 50.75, 30.5, 28.75, 38.625, ...","[3.0, 3.6015625, 5.65625, 1.2265625, 3.1328125...","[9.90625, 39.75, 9.75, 39.125, 10.0625, 57.0, ..."
2,821318.23,821318,23,2265.704834,0.396063,0.378523,0.392204,0.393676,2640933.500,"[9.78125, 12.28125, 26.9375, 9.15625, 2.804687...","[30.8125, 30.75, 30.875, 29.75, 51.25, 33.625,...","[3.21875, 1.78125, 5.765625, 2.9140625, 1.7343...","[31.3125, 40.25, 41.875, 30.75, 31.8125, 54.75..."
3,821318.27,821318,27,146.783783,0.143877,0.033444,0.144958,0.081382,2905792.000,"[2.3828125, 3.90625, 1.9921875, 0.857421875, 2...","[56.625, 10.625, 55.25, 56.625, 56.5, 47.75, 1...","[3.203125, 1.609375, 5.390625, 4.78125, 2.3906...","[39.875, 21.625, 39.75, 39.875, 14.03125, 31.5..."
4,821318.52,821318,52,386.725128,0.574187,0.237915,0.573510,0.250717,3662252.000,"[7.4375, 2.3046875, 9.5, 21.0, 6.375, 2.5, 2.9...","[28.9375, 27.5, 29.125, 28.875, 11.4375, 28.93...","[18.125, 37.375, 11.0, 10.78125, 10.65625, 6.6...","[28.75, 28.875, 30.5625, 29.0, 28.75, 28.75, 2..."
5,821318.78,821318,78,998.994019,0.181265,-0.604455,0.182779,5.716389,3326940.500,"[2.484375, 1.91015625, 1.77734375, 1.81640625,...","[20.125, 19.625, 16.4375, 20.5625, 27.6875, 38...","[6.578125, 7.265625, 14.71875, 8.03125, 3.1796...","[30.25, 44.5, 30.0, 30.5, 28.375, 22.8125, 52...."
6,821318.84,821318,84,187.254745,0.546767,-0.097338,0.548711,6.198726,2504340.000,"[2.5234375, 2.7890625, 1.35546875, 6.890625, 0...","[32.625, 32.375, 32.75, 56.875, 11.59375, 50.2...","[3.9140625, 8.5625, 1.8984375, 1.87109375, 4.1...","[53.875, 53.75, 25.75, 55.875, 29.8125, 11.812..."
7,821318.119,821318,119,140.490723,0.509423,-2.394012,0.505706,3.877012,3575588.250,"[3.4296875, 6.34375, 0.8203125, 2.3515625, 7.7...","[27.0625, 49.625, 32.25, 35.625, 26.6875, 27.3...",NaN,NaN
8,821318.128,821318,128,89.078133,0.260124,2.401048,0.255261,2.420712,2323491.000,"[2.7109375, 1.68359375, 1.45703125, 2.984375, ...","[52.0, 52.875, 51.0, 43.0, 21.75, 17.25, 56.37...","[2.71875, 0.2265625, 2.890625, 1.4609375, 1.37...","[18.125, 8.34375, 42.125, 26.5625, 18.3125, 17..."
9,821318.142,821318,142,354.145203,0.495758,-1.386395,0.492278,4.884024,3271002.500,"[2.8359375, 4.09375, 2.7578125, 1.5078125, 1.8...","[41.0, 26.9375, 42.125, 11.15625, 39.75, 29.56...","[1.921875, 0.2578125, 1.4375, 1.3671875, 1.503...","[42.5, 35.5, 41.625, 42.5, 42.5, 35.75, 50.875..."


# Protons
- does it work with protons?
- yes, but some files can be buggy

In [8]:
import sys

sys.path.append("..")

from magicdl import convert

In [24]:
error_sample_file = "/remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M1/GA_M1_za05to35_8_110000_Y_w0.root"
sample_file = "/remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M1/GA_M1_za05to35_8_110001_Y_w0.root"


In [25]:
o, e = convert.extract_calibrated_mc(sample_file, verbose=True)

Telescope 1 / Run 110001
Extracted 26 events from /remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M1/GA_M1_za05to35_8_110001_Y_w0.root
--- Overall params: ---
fTelescopeNumber 1
fRunNumber 110001
fNumTrig 26

--- Event params: ---
fEvtNumber (26,)
fEnergy (26,)
fZFirstInteraction (26,)
fTheta (26,)
fPhi (26,)
fTelescopeTheta (26,)
fTelescopePhi (26,)
fPhot (26, 1183)
MArrivalTime (26, 1183)
event_id (26,)



In [32]:
m1_filepath = "/remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M1/GA_M1_za05to35_8_110000_Y_w0.root"
m2_filepath = "/remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M2/GA_M2_za05to35_8_110000_Y_w0.root"

res = convert.merge_calibrated_mc(
    m1_filepath=m1_filepath,
    m2_filepath=m2_filepath,
    verbose=True,
)

Error extracting fPhot from /remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M1/GA_M1_za05to35_8_110000_Y_w0.root: cannot convert to RegularArray because subarray lengths are not regular (in compiled code: https://github.com/scikit-hep/awkward/blob/awkward-cpp-39/awkward-cpp/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)
Error extracting fPhot from /remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M2/GA_M2_za05to35_8_110000_Y_w0.root: cannot convert to RegularArray because subarray lengths are not regular (in compiled code: https://github.com/scikit-hep/awkward/blob/awkward-cpp-39/awkward-cpp/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)
Error extracting data from /remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M1/GA_M1_za05to35_8_110000_Y_w0.root
Error extracting data from /remote/lstdata01/jgreen/MAGIC_MC/protons/calibrated_M2/GA_M2_za05to35_8_110000_Y_w0.root


In [31]:
res